# Creating a feature matrix from a DB Query (Opto Sessions)

In [29]:
import sys
sys.path.append('/Users/Sabatini Lab/GitHub/mouse_bandit/data_preprocessing_code/opto_preprocessing')
sys.path.append('/Users/Sabatini Lab/GitHub/mouse_bandit')
import support_functions as sf
import numpy as np
import pandas as pd
import scipy as sp
import bandit_preprocessing_opto as bp
from sklearn import preprocessing
import sys
import os
%matplotlib inline 

## Retrieve names of sessions you want from the DB

In [30]:
#load in data base
db = pd.read_csv('/Users/Sabatini Lab/GitHub/mouse_bandit/mike_session_record.csv',index_col=0)

### Query all 90-10 sessions where performance > 0.7 block structure was 50 and had no laser trials

In [31]:
r = db[((db['Left Reward Prob'] == 0.9) |  (db['Right Reward Prob'] == 0.9))].copy()
r = r[r['p(high Port)'] > 0.5].copy()
r = r[r['Block Range Min'] == 50].copy()
r = r[r['Laser Stim Prob'] == 0.1].copy()

session_names = r['Session ID'].values 

In [32]:
r.tail(10)

,Owner,Session ID,Mouse ID,Date,Phase,Laser Stim Prob,Left Reward Prob,Right Reward Prob,Block Range Min,Block Range Max,No. Trials,No. Blocks,No. Rewards,p(high Port),Decision Window Duration,Min Inter-trial-interval,Left Solenoid Duration,Right Solenoid Duration
1090,mike,07282017_SOM44,SOM44,2017-07-28 00:00:00,2,0.1,0.9,0.1,50,50,257,3,181,0.75,2,1.0,40,40
1091,mike,07282017_SOM45,SOM45,2017-07-28 00:00:00,2,0.1,0.9,0.1,50,50,515,7,365,0.77,2,1.0,40,40
1097,mike,08012017_SOM41,SOM41,2017-08-01 00:00:00,2,0.1,0.1,0.9,50,50,545,8,432,0.87,2,1.0,40,40
1098,mike,08012017_SOM44,SOM44,2017-08-01 00:00:00,2,0.1,0.9,0.1,50,50,105,1,73,0.79,2,1.0,40,40
1105,mike,08032017_SOM41,SOM41,2017-08-03 00:00:00,2,0.1,0.1,0.9,50,50,289,4,230,0.91,2,1.0,40,40
1106,mike,08032017_SOM44,SOM44,2017-08-03 00:00:00,2,0.1,0.9,0.1,50,50,273,4,223,0.86,2,1.0,40,40
1107,mike,08032017_SOM45,SOM45,2017-08-03 00:00:00,2,0.1,0.1,0.9,50,50,536,8,423,0.87,2,1.0,40,40
1114,mike,08072017_SOM41,SOM41,2017-08-07 00:00:00,2,0.1,0.9,0.1,50,50,641,10,541,0.91,2,1.0,40,40
1115,mike,08072017_SOM44,SOM44,2017-08-07 00:00:00,2,0.1,0.9,0.1,50,50,457,7,381,0.92,2,1.0,40,40
1116,mike,08072017_SOM45,SOM45,2017-08-07 00:00:00,2,0.1,0.9,0.1,50,50,637,10,503,0.85,2,1.0,40,40


In [33]:
r.shape

(25, 18)

## load in csv files (from running exportTrials.m) added in 'Laser Given' column in trials for new data format following opto changes

In [34]:
'''
load in trial data
'''
columns = ['Elapsed Time (s)','Since last trial (s)','Trial Duration (s)','Port Poked','Right Reward Prob','Left Reward Prob','Reward Given','Laser Given']

root_dir = '/Users/Sabatini Lab/GitHub/mouse_bandit/data/mike_data/trial_data'

trial_df = []

for session in session_names:
    full_name = session + '_trials.csv'
    
    path_name = os.path.join(root_dir,full_name)
    
    trial_df.append(pd.read_csv(path_name,names=columns))

mouse_ids = r['Mouse ID'].values

## convert into 1 feature matrix

In [35]:
for i,df in enumerate(trial_df):
    
    curr_feature_matrix = bp.create_feature_matrix(df,10,mouse_ids[i],session_names[i],feature_names='Default')
    
    if i == 0:
        master_matrix = curr_feature_matrix.copy()
    else:
        master_matrix = master_matrix.append(curr_feature_matrix)
    

In [36]:
master_matrix.shape

(9858, 54)

In [37]:
master_matrix['Higher p port'].mean()

0.81953743152769321

In [38]:
master_matrix['Laser'].mean()

0.10093325218096977

## Stephenson et al acquired ~6500 trials per animal

In [39]:
master_matrix['Session ID'].unique()

array(['07122017_PV30', '07122017_SOM42', '07142017_SOM42',
       '07172017_SOM41', '07172017_SOM44', '07172017_SOM45',
       '07182017_SOM42', '07192017_SOM41', '07192017_SOM44',
       '07192017_SOM45', '07202017_SOM42', '07212017_SOM44',
       '07212017_SOM45', '07252017_SOM44', '07252017_SOM45',
       '07282017_SOM44', '07282017_SOM45', '08012017_SOM41',
       '08012017_SOM44', '08032017_SOM41', '08032017_SOM44',
       '08032017_SOM45', '08072017_SOM41', '08072017_SOM44',
       '08072017_SOM45'], dtype=object)

In [40]:
master_matrix.tail(10)

,Mouse ID,Session ID,Trial,Block Trial,Block Reward,Port Streak,Reward Streak,10_Port,10_Reward,10_ITI,...,1_Reward,1_ITI,1_trialDuration,0_ITI,0_trialDuration,Decision,Switch,Higher p port,Reward,Laser
617,SOM45,08072017_SOM45,628.0,56.0,46.0,4.0,4.0,0.520,0.0,1.0,...,0.0,1.0,1.543,0.236,1.370,0.0,0.0,1.0,1.0,0.0
618,SOM45,08072017_SOM45,629.0,57.0,47.0,5.0,5.0,0.443,0.0,1.0,...,0.0,1.0,1.370,0.520,2.271,0.0,0.0,1.0,1.0,0.0
619,SOM45,08072017_SOM45,630.0,58.0,48.0,6.0,6.0,0.547,0.0,1.0,...,0.0,1.0,2.271,0.443,2.057,0.0,0.0,1.0,1.0,0.0
620,SOM45,08072017_SOM45,631.0,0.0,49.0,7.0,7.0,0.445,0.0,1.0,...,0.0,1.0,2.057,0.547,3.809,0.0,0.0,0.0,0.0,0.0
621,SOM45,08072017_SOM45,632.0,1.0,0.0,8.0,-1.0,0.414,0.0,1.0,...,0.0,0.0,3.809,0.445,1.846,0.0,0.0,0.0,0.0,0.0
622,SOM45,08072017_SOM45,633.0,2.0,0.0,9.0,-2.0,0.624,1.0,0.0,...,0.0,0.0,1.846,0.414,2.524,1.0,1.0,1.0,1.0,0.0
623,SOM45,08072017_SOM45,634.0,3.0,1.0,1.0,1.0,0.456,0.0,1.0,...,1.0,1.0,2.524,0.624,1.911,0.0,1.0,0.0,0.0,0.0
624,SOM45,08072017_SOM45,635.0,4.0,1.0,1.0,-1.0,0.458,0.0,1.0,...,0.0,0.0,1.911,0.456,1.833,0.0,0.0,0.0,0.0,0.0
625,SOM45,08072017_SOM45,636.0,5.0,1.0,2.0,-2.0,0.451,0.0,1.0,...,0.0,0.0,1.833,0.458,23.542,1.0,1.0,1.0,1.0,0.0
626,SOM45,08072017_SOM45,637.0,6.0,2.0,1.0,1.0,0.339,0.0,1.0,...,1.0,1.0,23.542,0.451,1.421,1.0,0.0,1.0,1.0,0.0


## Save combined feature matrix  

In [41]:
master_matrix.to_csv(os.path.join(root_dir,'0.1opto_trials_9010_high.csv'))